In [342]:
from pathlib import Path

import click
import pandas as pd
from recsys.config import Config

import recsys.algorithms.last_hotel_algorithm.rec_last_hotel as rec
import recsys.algorithms.utils as utils

In [343]:
config = Config()

# calculate path to files
default_data_directory = config.data_path
data_directory = default_data_directory
test_csv = data_directory.joinpath('test.csv')
train_csv = data_directory.joinpath('train.csv')
subm_csv = data_directory.joinpath('submission_last_hotel.csv')

df_test = pd.read_csv(test_csv)
#df_train = pd.read_csv(train_csv)


df_target = utils.get_local_submission_target(df_test)

In [344]:
def last_element(l):
    if len(l) != 0:
        l = l[:-1]
    if len(l) != 0:   
        return l[::-1]
    else:
        return -1

In [345]:
df_grouped = df_test.groupby(['user_id', 'session_id'], as_index=False)['reference'].apply(list)
df_references = df_grouped.reset_index(name='references')

In [346]:
df_references['last_ref_hotel'] = df_references['references'].apply(lambda x: last_element([e for e in x if str(e).isdigit()]))

In [347]:
df_last_ref = df_references[df_references['last_ref_hotel'] != -1][['user_id', 'session_id', 'last_ref_hotel']]

In [348]:
df_target['impressions'] = df_target['impressions'].apply(utils.string_to_array)

/data/recsys-challenge-2019/recsys_env/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [349]:
df_joined = df_target.merge(df_last_ref, on=['user_id', 'session_id'], how='left')

In [350]:
def reorder(x):
    impressions = x['impressions'].copy()
    if type(x['last_ref_hotel']) == list:
        for last_ref_hotel in x['last_ref_hotel']:
            if str(last_ref_hotel) in impressions:
                impressions.remove(str(last_ref_hotel))
                impressions = [str(last_ref_hotel)] + impressions
    return impressions

In [351]:
df_joined['impressions_reordered'] = df_joined.apply(reorder, axis=1)

### Scoring

In [352]:
from recsys.submission.score_submission.functions import get_reciprocal_ranks

df_score = pd.DataFrame()

df_score['reference'] = df_joined['reference']

df_score['item_recommendations'] = df_joined['impressions_reordered'].copy()

df_score.apply(get_reciprocal_ranks, axis=1).mean()

0.6489613224696991